In [1]:
import scipy
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy import special
from scipy.optimize import minimize
from scipy.interpolate import interp1d

from mpmath import invertlaplace
from mpmath import *
mp.dps = 10; mp.pretty = True

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_2D(a, mu, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    while np.linalg.norm(x, 2) < a(rt):
        x += mu*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta = np.arctan2(x[1], x[0])   
    
    return ndt+rt, theta

In [3]:
def k(a, da, t, q, sigma=2):
    return 0.5 * (q - 0.5*sigma - da(t))

def psi(a, da, t, z, tau, q, sigma=2):
    kk = k(a, da, t, q, sigma)
    
    term1 = 1./(sigma*(t - tau)) * np.exp(- (a(t) + z)/(sigma*(t-tau)))
    term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
    term3 = da(t) - (a(t)/(t-tau)) + kk
    term4 = special.iv(q/sigma-1, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
    term5 = (np.sqrt(a(t)*z)/(t-tau)) * special.iv(q/sigma, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
    
    return term1 * term2 * (term3 * term4 + term5)

def ie_bessel_fpt(a, da, q, z, dt=0.1, T_max=2):
    g = [0]
    T = [0]
    g.append(-2*psi(a, da, dt, z, 0, q))
    T.append(dt)
    
    for n in range(2, int(T_max/dt)+2):
        s = -2 * psi(a, da, n*dt, z, 0, q)

        for j in range(1, n):
            s += 2 * dt * g[j] * psi(a, da, n*dt, a(j*dt), j*dt, q)

        g.append(s)
        T.append(n*dt)
        
    g = np.asarray(g)
    T = np.asarray(T)
    
    gt = interp1d(T, g)
    return gt

In [4]:
def HSDM_2D_likelihood(prms, RT, Theta):
    ndt = prms[1]
    mu = np.array([prms[2], prms[3]])
    
    if prms[0] <= 2:
        a = lambda t: prms[0]**2
        da = lambda t: 0
        fpt = ie_bessel_fpt(a, da, mu.shape[0], 0.0001, 
                            dt=0.01, T_max=max(RT))
    else:
        a = prms[0]
        nu = (mu.shape[0]-2)/2
        fixed = 2**nu * gamma(1+nu)
        fpt = lambda p: (a*sqrt(2*p))**nu/fixed *  1./besseli(nu, a*sqrt(2*p))
    
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.001:
            mu_dot_x0 = mu[0]*np.cos(theta)
            mu_dot_x1 = mu[1]*np.sin(theta)
            term1 = prms[0] * (mu_dot_x0 + mu_dot_x1)
            term2 = 0.5 * np.linalg.norm(mu, 2)**2 * (rt - ndt)
            
            if prms[0] < 2: 
                density = exp(term1 - term2) * fpt(rt - ndt)

                if 0.1**14 < density:
                    log_lik += -log(density)
                else:
                    log_lik += -log(0.1**14)
            else:
                density = exp(term1 - term2) * invertlaplace(fpt, rt-ndt, method='talbot')
                if 0.1**14 < density:
                    log_lik += -log(density)
                else:
                    log_lik += -log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
recovery_df = {'threshold_true': [],
               'threshold_estimate': [],
               'ndt_true': [],
               'ndt_estimate': [],
               'mu1_true': [],
               'mu1_estimate': [],
               'mu2_true': [],
               'mu2_estimate': []}

min_threshold = 0.5
max_threshold = 5

min_ndt = 0.1
max_ndt = 1

min_mu = -3.5
max_mu = 3.5

In [6]:
for n in tqdm(range(150)):
    threshold = np.random.uniform(min_threshold, max_threshold)
    a = lambda t: threshold
    ndt = np.random.uniform(min_ndt, max_ndt)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu)])
    
    
    recovery_df['threshold_true'].append(threshold)
    recovery_df['ndt_true'].append(ndt)
    recovery_df['mu1_true'].append(mu[0])
    recovery_df['mu2_true'].append(mu[1])
    
    RT = []
    Theta = []
    
    for i in range(250):
        rt, theta = simulate_HSDM_2D(a, mu, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    min_ans = minimize(HSDM_2D_likelihood,
                       args=(RT, Theta), 
                       x0=np.array([np.random.uniform(min_threshold, max_threshold),
                                    np.random.uniform(min_ndt, max_ndt), 
                                    np.random.uniform(min_mu, max_mu),
                                    np.random.uniform(min_mu, max_mu)]),
                       method='Nelder-Mead', 
                       bounds=[(min_threshold, max_threshold), (min_ndt, max_ndt),
                               (min_mu, max_mu), (min_mu, max_mu)])
    
    recovery_df['threshold_estimate'].append(min_ans.x[0])
    recovery_df['ndt_estimate'].append(min_ans.x[1])
    recovery_df['mu1_estimate'].append(min_ans.x[2])
    recovery_df['mu2_estimate'].append(min_ans.x[3])
    
recovery_df = pd.DataFrame(recovery_df)

100%|█████████████████████████████████████████████| 150/150 [13:16:53<00:00, 318.76s/it]


In [7]:
recovery_df

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate
0,0.984774,0.950179,0.348811,0.366270,0.586937,0.594571,0.568358,0.614045
1,2.520612,2.499590,0.449678,0.440309,2.062490,2.123062,0.138559,0.084358
2,2.923851,1.262899,0.593727,1.000000,-1.990057,-1.447678,-2.239259,-1.607796
3,4.583367,3.425051,0.759614,1.000000,2.445922,2.463563,-3.396105,-3.315296
4,4.513250,4.683140,0.541311,0.526335,-2.885679,-2.953010,-3.039149,-3.048749
...,...,...,...,...,...,...,...,...
145,4.916693,4.668455,0.918144,1.000000,-3.261182,-3.250676,2.528215,2.586069
146,3.551021,3.498266,0.598224,0.654416,-0.279484,-0.261066,-0.035729,-0.051001
147,0.947252,1.033352,0.309355,0.319648,2.506205,3.500000,2.422669,2.586650
148,2.248167,2.584724,0.610043,0.544736,-3.264861,-3.258140,1.964515,2.173543


In [8]:
recovery_df.corr()

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate
threshold_true,1.000000,0.901217,0.059525,0.111183,-0.075166,-0.063225,-0.024864,-0.027343
threshold_estimate,0.901217,1.000000,0.144723,-0.060221,-0.074257,-0.067413,0.051642,-0.013212
ndt_true,0.059525,0.144723,1.000000,0.731507,-0.161888,-0.162056,-0.008021,-0.025238
ndt_estimate,0.111183,-0.060221,0.731507,1.000000,-0.190098,-0.193531,-0.098665,-0.064724
mu1_true,-0.075166,-0.074257,-0.161888,-0.190098,1.000000,0.884214,-0.045050,0.004801
mu1_estimate,-0.063225,-0.067413,-0.162056,-0.193531,0.884214,1.000000,-0.048431,0.024404
mu2_true,-0.024864,0.051642,-0.008021,-0.098665,-0.045050,-0.048431,1.000000,0.882923
mu2_estimate,-0.027343,-0.013212,-0.025238,-0.064724,0.004801,0.024404,0.882923,1.000000


In [9]:
recovery_df.to_csv('Hybrid_2d_recovery.csv')